# Práctica 2 Aprendizaje y Selección de Modelos Minería de Datos 2020/21
Hecha por: **Cristian Stanimirov Petrov** y **Nikola Svetlozarov Dyulgerov**

# Pima Diabetes


## Preliminares 
Como siempre lo primero de todo será cargar los módulos que vamos a usar:

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer

# Local application
import md_grupos_practica2_utils as utils

In [ ]:
random_state = 265

## Carga de datos
Empezaremos por la base de datos de **Pima-Indians Diabetes** que utilizamos en la anterior práctica.

In [ ]:

target = "Outcome"

data = utils.load_data('../input/pima-indians-diabetes-database/diabetes.csv', target, index=None)

Comprobamos que se ha cargado correctamente con una muestra

In [ ]:
data.sample(5, random_state=random_state)

In [ ]:
data.shape

Como siempre, dividimos en variables predictoras (`X`) y variable clase (`y`)

In [ ]:
(X, y) = utils.divide_dataset(data, target)

Comprobamos las dos particiones 

In [ ]:
X.sample(5, random_state=random_state)

In [ ]:
y.sample(5, random_state=random_state)

Todo correcto. Pasemos a realizar la división de conjunto de entrenamiento y prueba con la técnica de holdout estratificado

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      train_size=0.7,
                                                      random_state=random_state)

Y de nuevo hacemos las comprobaciones necesarias antes de pasar a trabajar con los conjuntos de datos

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

## Selección de modelos
En esta sección lo que haremos será una evaluación individual de los modelos que genera cada algoritmo propuesto. Intentaremos encontrar los hiperparámetros que mejor se amoldan con el fin de seleccionar el modelo que más nos interesa. Además de ello comentaremos el funcionamiento y resultados de cada algoritmo, así como las salidas que vayamos obteniendo a lo largo del proceso.

Para llevar a cabo esta tarea haremos uso del algoritmo de búsqueda en *grid* que proporciona `scikit-learn`. Básicamente en lo que consiste es en un método por fuerza bruta. Realiza una búsqueda exhaustiva mediante validación cruzada probando todas las combinaciones de hiperparámetros que se le hayan proporcionado previamente.

Antes de empezar con la búsqueda en grid, debemos especificar el tipo de validación cruzada que vamos a emplear. En nuestro caso nos hemos decantado por una $5 x 10 - cv$ estratificada.

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

#### Vecinos más cercano
Empezamos con el algoritmo de los vecinos más cercanos. Primeramente creamos el **pipeline** con el procesamiento de datos que ya hicimos en la práctica pasada, solo que ahora añadimos como paso final el algoritmo para poder así evaluar los diferentes modelos que crea en base a los hiperparámetros. Estos últimos los establecemos acorde a nuestro conocimiento de la teoría, así como por las conclusiones sacadas del análisis exploratorio, ya que no tiene sentido perder el tiempo probando combinaciones absurdas. Una vez hecho todo esto invocaremos la función de búsqueda en grid que se nos proporciona con los parámetros correspondientes. En este último paso, es muy importante fijar las métricas de evaluación apropiadas para el problema dado,  así como indicar los hiperparámetros que debe usar el algoritmo.

**¡Aclaración!**
Antes, cuando hemos mencionado la creación de modelos, nos referimos en general y no a este primer algoritmo, ya que este en concreto no crea ninguno sino que utiliza toda la base de datos para hacer sus predicciones.

In [ ]:

column_transformer = make_column_transformer(("drop",["SkinThickness","Insulin"]),(SimpleImputer(missing_values = 0, strategy="mean"), ["Glucose","BloodPressure","BMI"]), remainder="passthrough")

pipeline = make_pipeline(column_transformer, KNeighborsClassifier())

weights = ["uniform", "distance"]
metrics = ["euclidean", "manhattan", "minkowski"]
n_neighbors = [5, 7, 9, 11, 13, 15, 17, 19, 21, 23]

k_neighbors_clf = utils.optimize_params(pipeline,
                                        X_train, y_train, cv,
                                        kneighborsclassifier__weights = weights,
                                        kneighborsclassifier__metric = metrics,
                                        kneighborsclassifier__n_neighbors=n_neighbors, scoring=['recall', 'roc_auc'], refit='roc_auc')

In [ ]:
X_train.shape

Para el algoritmo de los vecinos cercanos tenemos únicamente dos hiperparámetros que ajustar: **función distancia** y **número de vecinos**. Por defecto en el paquete de `sci-kit` están fijados a `minkowski` y `5`, respectivamente. De la teoría sabemos que la forma de medir la distancia tiene un impacto sobre los resultados obtenidos, en este caso la mejor solución encontrada es a través de `manhattan` que es de las maneras más simples de calcular la distancia entre dos puntos. Por otro lado, tenemos lo que se conoce como pesos, esto se refiere a la a como se realiza la predicción, hay dos posibilidades `uniform` todos los vecinos cercanos tienen la misma importancia o `distance` lo que da un peso proporcional a la inversa de la distancia desde el punto que se busca clasificar. En nuestro caso, la mejor opción es la de asignar el mismo peso a todos. Por otro lado, tenemos el ajuste del parámetro $k$ y para ello nos valemos de la opción secuencial, es decir, vamos probando valores en secuencia hasta un máximo de la raíz cuadrada de los ejemplos totales de nuestra base de datos $\sqrt{537} = 23.17$ Los valores que damos son impares a propósito de evitar posibles confusiones en situaciones de empate. En este caso el mejor valor encontrado es `19` que es un poco menor que el máximo propuesto. Sabemos de la teoría que un número pequeño de $k$ hace al clasificador muy sensible al ruido y también lo que ocurre es lo que conocemos como **underfitting**, es decir, se trata de un modelo muy simple e insuficiente para captar la información que nos dan las variables. Por eso directamente hemos saltado esos valores pequeños por debajo de 5 porque sabemos de antemano que por lo general dan malos resultados. Otra sería la situación si la base de datos fuese realmente pequeña, pero no es el caso. En cambio, un valor de $k$ muy alto suprime ese ruido diluyendo las fronteras de decisión, pero el problema que surge es el **sobre-ajuste** al conjunto de entrenamiento, de ahí que limitemos el incremento de su valor. 

En cuanto al método de evaluación nos hemos decantado por la métrica `roc_auc`, ya que es la que usamos en la práctica anterior. Cuanto más cerca sea la puntuación a uno, mejor es el modelo. Para contrastar esta parte y tener una visión más global, usaremos también el `recall` como otra alternativa que en este caso los resultados obtenido evaluados con esta métrica son bastante malos, ya que al igual que la otra métrica cuanto más se acerque el valor a uno mejor es y con esta giran entorno al `0.56` lo que está bastante lejos.

#### Arboles de decisión


Al igual que antes, creamos el **pipeline**, pero ahora como paso final lo que pasamos es el árbol de clasificación de `sci-kit`. Es lo mismo que haremos con los demás algoritmos. Muy importante establecer el `random_state` para que no haya variaciones en los resultados obtenidos.

In [ ]:
pipeline = make_pipeline(column_transformer, DecisionTreeClassifier(random_state=random_state))

criterion = ["gini", "entropy"]
max_depth = [ 2, 3, 4, 5, 6]
min_samples_split = [5, 10, 15, 20, 30] #afecta al max depth
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]

decision_tree_clf = utils.optimize_params(pipeline,
                                          X_train, y_train, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__min_samples_split=min_samples_split, #no aparece en el grid params, pero si en el display final
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha, 
                                          scoring=['recall', 'roc_auc'], 
                                          refit='roc_auc')

Si nos vamos a la página de `sci-kit` sobre los árboles de decisión veremos que hay muchos hiperparámetros que se pueden ajustar. Todos tienen que ver con el control de la ramificación de los árboles, ya que estos a pesar de ser muy fáciles de entender una vez construidos, tienden a sobreajustar fácilmente sobre el conjunto de datos de entrenamiento. Por nuestra parte, hemos decidido incluir un hiperparámetro más de los que se nos dan en la práctica. Con `min_samples_split` se controla la creación de nodos internos, es decir, es el mínimo número de ejemplos que debe haber en un nodo interno para que pueda ser divido. Esto implícitamente afectará a otros hiperparámetros como `max_depth`. En cuanto a la elección de valores de los hiperparámetros nos guiamos por lo que ya sabemos. Un valor de `max_depth` muy grande sobreajustará, así que lo mantenemos en valores que consideremos pequeños acorde a nuestras instancias y variables de la base de datos. Para `min_samples_split` suele rondar los valores hasta `40`, pero nos ha parecido un número elevado dado que el número de instancias no es demasiado alto. Tiene el mismo problema que otros hiperparámetros de los árboles de clasificación. Si su valor es pequeño, nos lleva a un sobre ajuste (muchas hojas con pocos ejemplos), pero si es un valor muy grande podemos provocar que el árbol no aprenda, es decir, hojas muy "generalistas" que se podrían dividir en hojas más específicas de las que extraer información relevante.

El mejor modelo ha usado como criterio para la división de nodos la entropía y no ha conseguido una profundidad muy grande, sino que se ha quedado en tres niveles. Esto quiere decir que el árbol no ha crecido en complejidad y tampoco ha sobreajustado si nos fijamos en la puntuación obtenida para el entrenamiento. Una de las razones por las que puede no haber creado otro nivel es que `min_sample_split` lo ha impedido. Otra cosa que notamos es que su valor tampoco es muy alto, cumpliendo nuesta hipótesis de no asignarle valores grandes. Por otro lado, tampoco se ha realizado demasiadas podas dado que el mejor valor de `ccp_alpha` es literalmente cero, lo cual indica que la complejidad del árbol creado no es elevada. Por último, nos queda comentar que de nuevo con la métrica de `recall` se obtienen puntuaciones bastante malas, en cambio con `roc_auc` rondamos los mismos valores que con el anterior algoritmo.

#### Adaptive Boosting (AdaBoost)

Comenzamos con los ensemebles. El procedimiento con el **pipeline** es exactamente el mismo que antes. Algo importante a tener en cuenta es el `base_estimator` que vamos a utilizar. Lo normal para este algoritmo es construir árboles sencillos, pero también podemos pasar otro tipo "learners". Sin embargo, como no los conocemos bien y se escapan de la envergadura de la práctica, tan solo usaremos los árboles de decisión como clasificadores que van a componer el ensemble.

In [ ]:
pipeline = make_pipeline(column_transformer, AdaBoostClassifier(random_state=random_state))

base_estimator = [DecisionTreeClassifier(random_state=random_state)]
n_estimators = [ 40, 50, 60, 70]
learning_rate = [0.1, 0.3, 0.9, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]

adaboost_clf = utils.optimize_params(pipeline,
                                     X_train, y_train, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     scoring=['recall', 'roc_auc'], 
                                     refit='roc_auc')

Como hiperparámetros principales o "directos" tenemos el **número de estimadores** que se utiliza para la construcción del ensemble y la **tasa de aprendizaje** que determina la contribución de los clasificadores a la combinación final. Tenemos que tener en cuenta la compensación que existe entre estos dos hiperparámetros. Por lo general cuando se usa un alto número de estimadores, se emplea una tasa muy baja. Esto no siempre implica buenos resultados, pero sí aumenta el coste computacional. Es muy común usar valores entre cero y uno para la tasa de aprendizaje, ya que de la otra manera (mayor que uno) suele dar problemas de sobre-ajuste. Hemos experimentado asignando varios valores bastante bajos, menores que `0.3`, y hemos obtenido mejores modelos, pero el tiempo de ejecución de la búsqueda en grid se disparaba mucho, así que hemos dejado los dos posibles extremos para ver el efecto que tiene y poder comparar los resultados obtenidos en un tiempo razonable. Por otro lado, están los hiperparámetros "indirectos" que vienen a ser los que necesita el `base_estimator`, es decir, los árboles de decisión que conforman el ensemble. Así que, como ya sabemos la teoría, la intención principal de este tipo de ensemble es unir clasificadores muy simples por lo que la profundidad de los árboles no debe ser muy grande. Y ya por otro lado podemos jugar con el criterio que se sigue para su construcción. No hemos incluido más parámetros para el `base_estimator` dado que el principal es la profundidad y añadir más combinaciones de hiperparámetros implica un incremento considerable del tiempo de ejecución. Queremos encontrar modelos representativos, pero siempre pensando también en el coste de estos además de su complejidad. 

Con todo esto, mirando los resultados obtenidos con el `grid search` podemos ver que los clasificadores utilizados tienen profundidad mínima, un solo nivel, y que en este caso ha funcionado la técnica de la tasa de aprendizaje baja con un número de estimadores razonable (el que viene por defecto). Por último comentar que de nuevo las puntuaciones con las dos métricas giran entorno a los valores visto con los dos algoritmos anteriores.

#### Bootstrap Aggregating (Bagging)


Al igual que para el anterior ensemble, tan solo usaremos los árboles de decisión como estimadores, aunque en este caso también podemos pasar otros como los vecinos más cercanos. 

In [ ]:
pipeline = make_pipeline(column_transformer, BaggingClassifier(random_state=random_state))

base_estimator = [DecisionTreeClassifier(random_state=random_state)]
n_estimators = [ 10, 50, 75, 100 ]
max_samples = [ 0.2, 1.0 , 1.2 ]
bootstrap = [ True, False ]
criterion = [ "gini", "entropy" ]

bagging_clf = utils.optimize_params(pipeline,
                                    X_train, y_train, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__max_samples=max_samples,
                                    baggingclassifier__bootstrap=bootstrap,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    scoring=['recall', 'roc_auc'], 
                                    refit='roc_auc')

A diferencia del boosting, en el **bagging** se deja a los árboles ramificar hasta el final. Es por eso, por lo que no controlamos su profundidad sino que los dejamos crecer hasta el final. Tan solo jugamos con el criterio con el que se construyen. En cuanto a los hiperparámetros "directos" de este ensemble tenemos el **número de estimadores** y el tamaño de los conjuntos de datos (boostraps) que usan los árboles para entrenar. Por lo general, a mayor número de clasificadores, mejores resultados se obtienen. La intuición nos dice que esto puede sugerir un sobre-ajuste, pero no es el caso ya que reducen las varianza al computar más votos entre los distintos clasificadores que se crean. El hiperparámetro que controla el tamaño de los bootstraps es `max_samples` que se expresa en porcentajes acorde al tamaño de la base de datos con la que se trata. Por defecto está fijado a uno lo que representa que estos conjuntos tienen el mismo tamaño que la base de datos. Normalmente se deja así, pero el uso de tamaños más pequeños puede suponer el incremento de la varianza de los árboles de decisión resultantes, lo cual se supone como una mejora en el rendimiento global. Por último, estos conjuntos de datos se pueden hacer con reemplazo o no, de ahí que probemos las dos alternativas para ver cual resulta ser mejor en este caso, aunque por lo general siempre se utiliza la  técnida del reemplazo. Dicho esto, podemos comprobar que nuestras hipótesis se cumplen dado que los conjuntos de datos tiene un tamaño de 20 % de 537 = 107.4 instancias lo cual es una porción significativa inferior al total del conjunto de entrenamiento y que se han construido con el método de reemplazo. Por otro lado tenemos que cuantos más árboles se crean mejor, se podría ver en qué número empieza a estabilizarse la puntuación para poder elegir el número perfecto de estos para este problema en concreto. De momento nos quedaremos con los 100 que hemos establecido.

#### Random Forest

In [ ]:
pipeline = make_pipeline(column_transformer, RandomForestClassifier(random_state=random_state))

n_estimators = [ 50, 100, 150 ]
max_features = ["sqrt", "log2"]
max_depth = [ None, 5, 10]
criterion = ["gini", "entropy"]

random_forest_clf = utils.optimize_params(pipeline,
                                          X_train, y_train, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__n_estimators=n_estimators,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__max_depth=max_depth,
                                          scoring=['recall', 'roc_auc'], 
                                          refit='roc_auc')

De nuevo nos encontramos que debemos fijar el **número de estimadores** y al igual que el resto de ensembles la regla general (no necesariamente siempre funciona) es que cuantos más mejor, pero ya sabemos que esto tiene un coste de tiempo y memoria considerables. Otro hiperparámetro importante para la creación de los árboles es la **cantidad de variables** que usarán, para ello se puede emplear como máximo la raíz cuadrada  o el logaritmo del total  de variables que componen la base de datos. Aparte de estos dos hiperparámetros que digamos son los más relevantes del ensemble, hay otros "secundarios" que afectan a cómo se construyen los árboles de decisión. En nuestro caso, nos hemos decantado solamente por ver qué ocurre si controlamos la profundidad de estos, aunque no es una práctica común ya que normalmente se deja que produzca todas las ramificaciones a pesar de sobreajustar al conjunto de datos. Sabiendo esto, la regla de cuantos más clasificadores mejor ha funcionado a la perfección. Como algo interesante sería ver en qué momento la puntuación se estabiliza y también hacer un balance si merece asumir el coste computacional que conlleva la utilización de tantos árboles. Lo que nos ha sorprendido es que el mejor modelo resultante tiene relativamente poca profundidad, esto se puede deber a que al limitar el número de variables de los árboles y al trabajar con una base de datos no muy grande, no nos resulte de provecho ramificar hasta el final, ya que solamente produciríamos un sobre-ajuste innecesario. De ahí que con árboles más pequeños se extrae mayor información.

#### Gradient Boosting

In [ ]:
pipeline = make_pipeline(column_transformer, GradientBoostingClassifier(random_state=random_state))

n_estimators = [ 50, 100, 150 ]
learning_rate = [0.01, 0.05, 0.1]
subsample = [0.2, 0.8, 1.0]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]

gradient_boosting_clf = utils.optimize_params(pipeline,
                                              X_train, y_train, cv,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__subsample=subsample,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              scoring=['recall', 'roc_auc'], 
                                              refit='roc_auc')

De nuevo nos encontramos con **número de estimadores** con lo que seguiremos la misma lógica hasta ahora con los ensembles, siempre teniendo cuidado con el coste temporal que implica crear muchos árboles. Tenemos también la **tasa de aprendizaje** que para este caso concreto hay una norma general y es que siempre se suelen fijar valores por debajo de `0.1`. Esto se debe a que si damos valores más altos que este, los árboles que se van añadiendo al modelo hacen menos correcciones y crecen rápidamente lo que acaba resultando en un sobreajuste final. Por lo que nuestra intención es relantizar ese crecimiento y llegar a un punto de balance con el número de estimadores. Por otro lado, jugamos con el tamaño de los conjuntos de datos con los que se construyen los árboles, algo similar a los `bootstrap sets`, por lo que hemos encontrado es que suelen funcionar mejor conjuntos ligeramente inferiores respecto al tamaño de la base de datos con la que se trabaja. Y ya los hiperparámetros que quedan son los encargados de darles características a los árboles de decisión. Como siempre nos quedamos con el **criterio** con el que se contruyen, ahora es diferente al índice gini y la entropía dado que no clasificamos sino que calculamos residuos o errores, y también su profundidad. Para esta último, suelen darse valores pequeños, ya que si los ramificamos enteramente al irlos agregando al modelo final rápidamente producirría sobre ajuste y lo que queremos es que se "fijen" en las variables más relevantes.

Viendo los resultados obtenidos, la teoría ha dado en el clavo. El mejor modelo se compone del máximo número de estimadores que se ha asignado, el `subsample` es ligeramente inferior al número de instancias totales, los árboles que se han ido agregando al modelo son simples (un nivel solamente) y ha triunfado una tasa de aprendizaje en el límite de lo que se suele establecer. Un siguiente paso que se podría dar es encontrar ese balance entre `n_estimators` y `learning_rate` del que hemos hablado, pero para ello debemos asumir el coste computacional que conlleva incrementar un hiperparámetro y rebajar el otro respectivamente. En nuestro caso, hemos intentado limitar un poco estos, ya que a partir de 150 estimadores el tiempo aumentaba considerablemente, pero sabemos que podría mejorar aun más la puntuación.

#### Histogram Gradient Boosting

Se trata de un modelo experimental que son más rápidos que el **Gradient Boosting** anterior cuando el número de instancias ronda un magnitud mayor a los diez mil. En otras palabras, está optimizado para bases de datos enormes comparadas con la nuestra.

In [ ]:
pipeline = make_pipeline(column_transformer, HistGradientBoostingClassifier(random_state=random_state))

loss = ["binary_crossentropy", "categorical_crossentropy"]
learning_rate = [0.01, 0.05, 0.1]
max_iter = [50, 100, 130]
min_samples_leaf = [15, 25, 35]

hist_gradient_boosting_clf = utils.optimize_params(pipeline,
                                                   X_train, y_train, cv,
                                                   histgradientboostingclassifier__loss=loss,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_iter=max_iter,
                                                   histgradientboostingclassifier__min_samples_leaf=min_samples_leaf,
                                                   scoring=['recall', 'roc_auc'], 
                                                   refit='roc_auc')

No hay mucho escrito aun sobre este ensemble, así que nos hemos guiado principalmente por lo que hay en la página de `sci-kit`. Se trata de una versión optimizada del **Gradient Boosting** para bases de datos muy grandes, es por eso que muchos de sus hiperparámetros se mantienen, en cambio otros no y aparecen otros nuevos. La **tasa de aprendizaje** sigue estando, así que seguiremos la misma estrategia que antes de asignar valores menores a `0.1`. El **número de estimadores** se ha convertido en `max_iter` y viene a controlar las iteraciones que realiza el proceso de boosting. Su objetivo es en parte mantener en unos márgenes razonables el coste computacional que conlleva su ejecución. Hay más hiperparámetros como este que tienen la función de controlar la condición de parada del ensemble, pero solo nos centraremos en este en concreto. Y ya lo que quedarían son los hiperparámetros encargados de controlar las caracerísticas de los árboles que se van construyendo. Antes usabamos `max_depth` como parámetro principal, pero esta vez emplearemos `min_samples_leaf`, ya que en la documentación expresa explícitamente que para bases de datos pequeñas como la nuestra es necesario controlarlo para evitar que se creen árboles poco profundos. El valor por defecto es 20, así que hemos dado valores en un rango entorno a este valor.

Mirando los resultados del `grid-search` nos damos cuenta la función de pérdida que mejor ha funcionado es la `binary_crossentropy` y esto se debe principalmente a que en esta base de datos el número de clases es dos así que era un tanto evidente que daría mejor puntuación que la categórica que se emplea para problemas multiclase. La tasa de aprendizaje es la mínima que hemos establecido, así que cumple con lo esperado. Por último, el valor mínimo de instancias en las hojas de los árboles ronda entorno al valor por defecto. Una cosa que llama la atención es que con valores por debajo de estos aparecen `NaN`, algo que no entendemos del todo porque puede ocurrir.

## Construcción y validación del modelo final
Una vez hemos optimizado los hiperparámetros de todos los modelos resultantes de los algoritmos estudiados, procedemos a encontrar el modelo final en base a su medida de rendimiento al generalizar y en caso de empates nos guiaremos por la navaja de Ockham mirando el coste computacional o la complejidad de interpretación/construcción. Para medir el rendimiento usaremos el conjunto de datos de prueba que hemos aislado de la validación cruzada al principio del proceso. 

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
utils.evaluate_estimators(estimators, X_test, y_test)


Con un rápido vistazo se puede ver que los mejores clasificadores han sido los ensembles con cierta ventaja respecto los demás. Al igual que antes usamos diferentes métricas de evaluación del rendimiento de los clasificadores. En este caso, el **Gradient Boosting** es el que mejor ha funcionado si nos fijamos en el `roc_auc_score` que es la métrica que hemos elegido como predeterminada en este problema. Sin embargo, si nos fijamos en el `recall_score` el vencedor sería su variante optimizada **Histogram Gradient Boosting**, pero con esta métrica los resultados parecen no ser del todo buenos, ya que aun hay margen significativo de mejora para todos los clasificadores. Al no haber empate, nos quedamos directamente con el claro vencedor, **Gradient Boosting** como modelo final.

# Wisconsin

## 2. Acceso y almacenamiento de datos

Cargamos nuestro conjunto de datos, en este caso el de `wisconsin` especificando que variable corresponde al identificador de las instancias del conjunto de datos y cual corresponde a la variable clase.

In [ ]:
# filepath = "../input/wisconsin.csv"
index = "id"
target = "diagnosis"

data = utils.load_data("../input/breast-cancer-wisconsin-data/data.csv", target, index)

Veamos un ejemplo de muestra aleatoria con la función `sample` para ver que se ha cargado correctamente.

In [ ]:
data.sample(5, random_state=random_state)

Como podemos observar en la muestra, tenemos una columna sin nombre (`Unnamed: 32`) que no tiene datos para niguna de las instancias, por lo tanto la eliminaremos puesto que no nos aporta nada.

In [ ]:
del data["Unnamed: 32"]
data.sample(5, random_state=random_state)

Vamos a separar nuestro conjunto de datos en dos subconjuntos, por un lado las variables predictoras y por otro lado la variable objetivo.

In [ ]:
(X,y) = utils.divide_dataset(data, target = target)

Y comprobamos que se haya realizado completamente. Primero, las variables predictoras:

In [ ]:
X.sample(5, random_state=random_state)

Y por otro la variable clase

In [ ]:
y.sample(5, random_state=random_state)

Realizamos el holdout para poder tener un conjunto de entrenamiento y por otro lado uno de prueba. Este último solo lo usaremos al final del proceso, mientras tanto el aprendizaje de modelo se hará con el de entrenamiento.

In [ ]:
train_size = 0.7
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=random_state,
                                                      train_size=train_size)

Comprobamos que se han separado correctamente:

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

Y después el conjunto de datos de prueba:

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

## Selección de modelos

Al igual que hemos realizado para *Pima-Indians Diabetes* en esta sección haremos una evaluación individual de los modelos buscando la mejor combinación de hiperparámetros para cada modelo.

Utilizaremos la validación cruzada usada anteriormente.

### Vecinos más cercanos

Como comentamos en el ejemplo de **Pima-Indians Diabetes** creamos el pipeline con el procesamiento de datos al cuál le añadiremos como paso final el algoritmo para evaluar los diferentes modelos en base a los hiperparámetros. En este caso el **pipeline** tiene más transformadores que ejecutar, pero la metódica de trabajo es exactamente la misma.

In [ ]:
column_transformer = make_column_transformer(("drop",["area_mean", "perimeter_mean", "area_se", "perimeter_se", 
                                                      "area_worst", "perimeter_worst", "concavity_mean", "compactness_mean", 
                                                      "concavity_se", "compactness_se", "concavity_worst", "compactness_worst"]), 
                                             remainder="passthrough")

discretizer = KBinsDiscretizer(n_bins=2, strategy="uniform")

pipeline = make_pipeline(column_transformer, discretizer, KNeighborsClassifier())

weights = ["uniform", "distance"]
metrics = ["euclidean", "manhattan", "minkowski"]
n_neighbors = [3, 5, 7, 9, 11, 13, 15, 17, 19]

k_neighbors_clf = utils.optimize_params(pipeline,
                                        X_train, y_train, cv,
                                        kneighborsclassifier__weights = weights,
                                        kneighborsclassifier__metric=metrics,
                                        kneighborsclassifier__n_neighbors=n_neighbors, scoring=['recall', 'roc_auc'], refit='roc_auc')

In [ ]:
X_train.shape

Los modelos resultantes del algoritmo de los vecinos cercanos mantenemos los mismos valores para los hiperparámetros a excepción del número de vecinos a buscar, es decir **k**. Debemos dar unos valores acordes a este nuevo conjunto de datos, usamos de nuevo una secuencia de valores impares, pero esta vez con máximo de `19` que se obtiene de $\sqrt{398} = 19.95$.  

Igual que el anterior caso, la mejor opción es asignar el mismo peso a todos: `uniform`. En cambio, la manera de medir la distancia que mejor nos conviene es la `euclidean` por lo que vemos efectivamente es que la función de distancia tiene diferente impacto dependiendo del problema del que se trata y debe elegirse cuidadosamente.

### Árboles de decisión

Volvemos a crear el **pipeline**, pero esta vez le pasamos como paso final el árbol de clasificación.

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer, DecisionTreeClassifier(random_state=random_state))

criterion = ["gini", "entropy"]
max_depth = [ 3, 4, 5, 6, 7]
min_samples_split = [10, 15, 20, 30, 40] #afecta al max depth
ccp_alpha = [0.0, 0.1, 0.2]

decision_tree_clf = utils.optimize_params(pipeline,
                                          X_train, y_train, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__min_samples_split=min_samples_split,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha, 
                                          scoring=['recall', 'roc_auc'], 
                                          refit='roc_auc')

El mejor modelo de árboles de decisión ha usado como criterio de división de nodos la ganancia de información y ha conseguido una profundidad de seis niveles, esto puede deberse a que tenemos un mayor número de variables de mayor relevancia y conseguie ramificar más que en el anterior problema. Por otro lado, en este caso hemos obtenido un `min_samples_split` algo más grande que en el **Diabetes**, pero, al igual que este, hemos obtenido un `ccp_alpha` nulo, por lo que no se ha realizado demasiadas podas.

### Adaptive Boosting (AdaBoost)

Creamos el nuevo **pipeline** y probamos las diferentes combinaciones.

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer, AdaBoostClassifier(random_state=random_state))

base_estimator = [DecisionTreeClassifier(random_state=random_state)]
n_estimators = [40, 50, 60, 70]
learning_rate = [0.1, 0.3, 0.9, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]

adaboost_clf = utils.optimize_params(pipeline,
                                     X_train, y_train, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     scoring=['recall', 'roc_auc'], 
                                     refit='roc_auc')

Analizando los resultados obtenidos podemos ver que el ensemble utiliza árboles de la profundidad mínima, un solo nivel, lo cual corrobora la teoría de usar estimadores muy simples para la construcción del modelo final. Por otro lado, se ha encontrado un balance entre la tasa de aprendizaje y el número de estimadores de `0.1` y `50`, respectivamente. Este ejemplo se sale de lo normal en cuanto al número de estimadores, ya que rompe la regla de cuantos más, mejor, y esto puede ser causa directa del valor asignado a la tasa de aprendizaje.

### Boosting Aggregation (Bagging)

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer, BaggingClassifier(random_state=random_state))

base_estimator = [DecisionTreeClassifier(random_state=random_state)]
n_estimators = [ 10, 50, 75, 100 ]
max_samples = [ 0.2, 1.0 , 1.2 ]
bootstrap = [ True, False ]
criterion = [ "gini", "entropy" ]

bagging_clf = utils.optimize_params(pipeline,
                                    X_train, y_train, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__max_samples=max_samples,
                                    baggingclassifier__bootstrap=bootstrap,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    scoring=['recall', 'roc_auc'], 
                                    refit='roc_auc')

 Los hiperpárametros obtenidos son totalmente iguales a los resultantes derivados del problema de **Diabetes**, exceptuando que en vez de 100 estimadores emplea solamente 75. Esto es un claro indicador de que la teoría nos da un apoyo sólido en cuanto a la asignación de valores para los hiperparámetros por probar, independientemente del tipo de problema. Eso sí, es muy importante saber qué cosas específicas retocar para cada caso con el fin de obtener mejores resultados.

### Random Forest

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer,RandomForestClassifier(random_state=random_state))

n_estimators = [ 50, 100, 150 ]
max_features = ["sqrt", "log2"]
max_depth = [ None, 5, 10]
criterion = ["gini", "entropy"]

random_forest_clf = utils.optimize_params(pipeline,
                                          X_train, y_train, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__n_estimators=n_estimators,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__max_depth=max_depth,
                                          scoring=['recall', 'roc_auc'], 
                                          refit='roc_auc')


Como podemos observar para elegir la cantidad de variables que usarán los árboles se usa la raíz del total de variables, lo cual es lo más típico. La profundidad obtenida es de `5` lo cual es relativamente poco profundo en comparación a un árbol totalmente ramificado. Y por último, volvemos a obtener un número alto de estimadores cumpliéndose una vez más la regla de oro de los ensembles.

### Gradient Boosting

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer, GradientBoostingClassifier(random_state=random_state))

n_estimators = [ 50, 100, 150 ]
learning_rate = [0.01, 0.05, 0.1]
subsample = [0.2, 0.8, 1.0]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]

gradient_boosting_clf = utils.optimize_params(pipeline,
                                              X_train, y_train, cv,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__subsample=subsample,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              scoring=['recall', 'roc_auc'], 
                                              refit='roc_auc')

Al igual que en **Diabetes**, la teoría vuelve a dar en el clavo. Obtenemos el máximo de estimadores que se ha asignado, un subsample ligeramente inferior al número de instancias, los árboles esta vez tienen una profundidad de `2` y tenemos una tasa de aprendizaje en el límite de lo que suele establecer: `0.1`. Con todo esto podemos volvemos a recalcar la importancia de saber cómo funcionan los algoritmos por dentro con el fin de tener una intuición base sobre los resultados a obtener.

### Histogram Gradient Boosting

Como esta version del **Gradient Boosting** realiza un discretizado interno, no permite el uso de discretizadores en pasos previos del **pipeline** por lo que hay dos opciones, o bien eliminar nuestros discretizador, o bien la opción que hemos elegido para este caso que implica el uso de **DenseTransformer**. Básicamente lo que hace es convertir `sparse_matrix` (matriz llena de ceros) en `dense_matrix` para que pueda ejecutarse correctamente.

In [ ]:
from mlxtend.preprocessing import DenseTransformer
to_dense = DenseTransformer()

In [ ]:
pipeline = make_pipeline(column_transformer, discretizer, to_dense, HistGradientBoostingClassifier(random_state=random_state))

loss = ["binary_crossentropy", "categorical_crossentropy"]
learning_rate = [0.01, 0.05, 0.1]
max_iter = [50, 100, 130]
min_samples_leaf = [15, 25, 35]

hist_gradient_boosting_clf = utils.optimize_params(pipeline,
                                                   X_train, y_train, cv,
                                                   histgradientboostingclassifier__loss=loss,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_iter=max_iter,
                                                   histgradientboostingclassifier__min_samples_leaf=min_samples_leaf,
                                                   scoring=['recall', 'roc_auc'], 
                                                   refit='roc_auc')

Como es lógico, la función de pérdida que mejor funciona es la `binary_crossentropy` ya que este problema tiene solo dos clases. De nuevo el "número de estimadores" es el más alto posible, sin embargo la tasa de aprendizaje baja respecto al **Gradient Boosting**. En cuanto a la partición de nodos de los árboles de decisión el mejor valor obtenido es ligeramente más alto de lo esperado. Esto implica de forma un poco más indirecta que si miramos por ejemplo `max_depth` de que los árboles construidos 
son más simples dado que sus nodos acaparan mas ejemplos.

## Construcción y validación del modelo final

Una vez hemos optimizado los hiperparámetros de todos los modelos resultantes procedemos al igual que antes a la parte final de encontrar el modelo final en base a su medida de rendimiento. De nuevo en caso de empate nos guiaremos por la navaja de Ockham.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
utils.evaluate_estimators(estimators, X_test, y_test)


De un plumazo podemos observar que las puntuaciones para este problemas son mucho más altas que en el anterior caso. Como ya hemos comentado, la métrica predefinida es `roc_auc_score`así que será en la cual nos fijemos primordialmente. Dicho esto, el claro vencedor es el modelo de los **vecinos más cercanos** que gana con cierta ventaja a los restantes, no sólo en puntuación sino en su simplicidad. Algo que nos ha sorprendido totalmente, dado que sabemos el potencial ofrecido por los ensembles para cualquier tipo de problema. Ahora bien, si nos fijamos en la puntuación obtenida con la métrica del `recall` vemos que la cosa no está del todo clara, ya que hay múltiples empates. Eso sí, el rendimiento de todos sigue estando ligeramente por debajo que con nuestra métrica establecida. En ese caso nos quedaríamos con el **árbol de decisión** ya que es un modelo mucho más fácil de entender y de explicar que un ensemble.

# Comentarios de un kernel
Hemos decidido separar esta parte de la práctica para recortar la longitud del cuaderno y ganar un poco de claridad en cuanto a las explicación. Este enlace te lleva a la reproducción del [kernel](https://www.kaggle.com/nikoladyulgerov/titanic-getting-better-eda-top-14-4932eb) que hemos elegido.

